In [1]:
import pandas as pd
import datetime
import warnings
import pymssql
import re
warnings.filterwarnings("ignore")  # 忽略UserWarning兼容性警告
now = datetime.date.today().strftime("%Y-%m-%d")
server = "120.253.186.206:12513"
user = "baseline2"
password = "qwert123!@#"
database = "Lichi"  

# 建立数据库连接
conn = pymssql.connect(server=server, user=user, password=password, database=database)
discount = float('0.83')
Discount_ye_now = float('0.76')
Discount_MMM_now = float('1.0')
Discount_SB_now = float('0.74')


df_jp = pd.read_excel(f'C:/Users/15516/Desktop/数据处理/竞对数据/比对目录——2023.07.05版.xlsx',sheet_name='比价明细')
df_MMM_now = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/mmm/梅苗苗{now}.csv')
df_YE_now  = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/牙E在线/牙E在线{now}.csv',encoding='GBK')
df_SB_now  = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/松佰/松佰-{now}.csv')

# 三地比价表

## 函数配置

In [2]:
def deal_with(area, discount):
    
    discount = discount
    
    # 读取相关数据文件
    df_cp = pd.read_excel(f'C:/Users/15516/Desktop/数据处理/励齿数据/{now}/{area}产品详情.xlsx')
    df_kc = pd.read_excel(f'C:/Users/15516/Desktop/数据处理/励齿数据/{now}/{area}库存.xlsx')
    df_jg = pd.read_excel(f'C:/Users/15516/Desktop/数据处理/励齿数据/{now}/{area}采购价格.xlsx')
    
    # 根据地区设置 lichi 变量
    area_mapping = {
        '上海': 'lichi',
        '北京': 'lichi_beijing',
        '广州': 'lichi_guangzhou'
    }
    lichi = area_mapping.get(area, '')
    
    # 构建查询语句，引用不同的 lichi 变量
    query_finalPrice = f'''
    SELECT cinvcode, iTaxUnitPrice
    FROM (
        SELECT cinvcode, iTaxUnitPrice, RANK() OVER (PARTITION BY cinvcode ORDER BY dInputDate DESC) AS rank
        FROM {lichi}..pu_purordermaster ma
        INNER JOIN {lichi}..PU_PurOrderDetail de ON ma.pomid = de.pomid
        WHERE
            iTaxUnitPrice NOT IN ('0', '0.01', '0.1')
            AND Status IN ('已审核', '关闭')
            AND NOT EXISTS (SELECT 1 FROM {lichi}..vendor WHERE ma.cVenCode = cVenCode AND cVenName LIKE '%励齿医疗%')
            AND igQuantity != 0
    ) AS a
    WHERE a.rank = 1;
    '''
    
    # 执行查询
    df_fp = pd.read_sql(query_finalPrice, conn)
    
    # 判断到手价是否为0或为空，根据条件计算到手价格
    df_cp['到手价格'] = df_cp.apply(lambda row: discount * row['价格'] if (row['到手价'] == 0 or pd.isnull(row['到手价'])) else row['到手价'], axis=1)
    
    # 选择所需的列
    df_cp = df_cp[['SKU编码', '名称', '是否上架', '价格', '到手价格']]
    
    # 执行左连接操作
    df_merged = pd.merge(df_cp, df_kc, left_on='SKU编码', right_on='励齿编码', how='left').merge(df_jg, left_on='SKU编码', right_on='励齿编码', how='left').merge(df_fp, left_on='SKU编码', right_on='cinvcode', how='left')
    
    # 选择所需的列，并对数值列进行舍入
    
    df_merged[['价格', '到手价格', '含税采购价', 'iTaxUnitPrice']] = df_merged[['价格', '到手价格', '含税采购价', 'iTaxUnitPrice']].apply(lambda x: round(x, 2), axis=1)
        
        
    df_merged = df_merged[['SKU编码','名称','是否上架','价格','到手价格','励齿库存','含税采购价','iTaxUnitPrice']]
    
    # 修改列名
    new_columns = ['SKU编码', f'{area}名称', f'{area}类型', f'{area}商城价格', f'{area}到手价格', f'{area}库存量', f'{area}采购价', f'{area}最后采购价格']
    df_merged.columns = new_columns
    
    return df_merged


def deal_with_MMM(df, Discount):
    """
    处理梅苗苗数据的函数

    参数:
    - df: 待处理的DataFrame
    - Discount: 折扣

    返回值:
    - 处理后的DataFrame
    """

    # 清除型号列中的空格
    df['型号'] = df['型号'].str.strip()

    # 合并品名和型号列，并在型号列为空时只保留品名
    df['品型'] = df['品名'] + df['型号'].fillna('')

    # 根据折扣价和价格列判断折扣价格列的值
    df['折扣价格'] = df.apply(lambda row: row['折扣价'] if pd.notnull(row['折扣价']) and row['折扣价'] != 0 else row['价格'] * Discount, axis=1)

    # 重新排列列的顺序
    df = df[['产品编码', '品型', '折扣价格', '标签']]
    
    # 重命名列名
    new_columns = ['梅苗苗编码', '梅苗苗品名', '梅苗苗.折扣价格', '梅苗苗.标签']
    df.columns = new_columns
    
    return df



def deal_with_YAE(df, Discount):
    """
    对DataFrame进行处理，添加折扣价格列，并返回处理后的DataFrame。

    参数：
    df (pandas.DataFrame): 要处理的DataFrame。
    Discount (float): 折扣值。

    返回：
    pandas.DataFrame: 处理后的DataFrame。
    """

    # 去除型号列中的空格
    df['型号'] = df['型号'].str.replace(' ', '')

    # 拼接链接、型号和商品编号列为品型列
    df['品型'] = df['链接'].astype(str) + df['型号'].astype(str) + df['商品编号'].astype(str)
    df['牙E编码'] = df['链接'].astype(str) + df['型号'].astype(str)

    # 计算折扣价格列
    df['折扣价格'] = df['价格'] * Discount 

    # 重新选择列，只保留牙E编码、品型和折扣价格列
    df = df[['牙E编码', '品型', '折扣价格']]

    # 重命名列名
    new_columns = ['牙E编码', '牙E在线品型', '牙E在线.折扣价格']
    df.columns = new_columns

    return df



def deal_with_SONGBAI(df, discount):
    """
    处理松佰数据的函数，替换价格中的"议价"和"￥"，根据折扣计算到手价格，并选择指定的列返回处理后的DataFrame。

    参数：
    - df (pandas.DataFrame): 要处理的DataFrame。
    - discount (float): 折扣值。

    返回：
    - pandas.DataFrame: 处理后的DataFrame。
    """

    # 替换商城价格中的"议价"和"￥"，并转换为浮点数
    df['商城价格'] = df['商城价格'].apply(lambda x: re.sub(r'(议价|￥)', lambda m: '0' if m.group(0) == '议价' else '', str(x))).astype(float)

    # 替换到手价格中的"议价"和"￥"，并转换为浮点数
    df['到手价格'] = df['到手价格'].apply(lambda x: re.sub(r'(议价|￥)', lambda m: '0' if m.group(0) == '议价' else '', str(x))).astype(float)

    # 根据条件计算到手价格
    df['到手价格'] = df.apply(lambda row: discount * row['商城价格'] if (row['到手价格'] == 0 or pd.isnull(row['到手价格'])) else row['到手价格'], axis=1)

    # 选择指定的列
    df = df[['商品编码', '商品名称', '到手价格', '厂家直发']]
    
    # 重命名列名
    new_columns = ['松佰编码', '松佰名称', '松佰商城.折扣价格', '松佰商城.标签']
    df.columns = new_columns

    return df




## 处理三地比价表数据

In [3]:
df_sh = deal_with('上海', discount)  # 获取上海地区的数据
df_bj = deal_with('北京', discount)  # 获取北京地区的数据
df_gz = deal_with('广州', discount)  # 获取广州地区的数据

# 合并SKU编码
df = pd.concat([df_sh['SKU编码'], df_bj['SKU编码'], df_gz['SKU编码']]).drop_duplicates().reset_index(drop=True)
df = pd.DataFrame({'SKU编码': df})

# 根据SKU编码进行左连接操作
df = pd.merge(df, df_sh, on='SKU编码', how='left').merge(df_bj, on='SKU编码', how='left').merge(df_gz, on='SKU编码', how='left')

# 重新组织列的顺序
df = df[['SKU编码', '上海名称', '北京名称', '广州名称', '上海类型', '北京类型', '广州类型', '上海商城价格', '北京商城价格', '广州商城价格', '上海到手价格', '北京到手价格', '广州到手价格', '上海库存量', '北京库存量', '广州库存量', '上海采购价', '北京采购价', '广州采购价', '上海最后采购价格', '北京最后采购价格', '广州最后采购价格']]


## 输出三地比价表

In [4]:
df.to_excel(f'C:/Users/15516/Desktop/三地比价{now}.xlsx',index = False)

# 竞品比价表

In [5]:
## 梅苗苗
df_mmm =  deal_with_MMM(df_MMM_now,Discount_MMM_now)

In [6]:
## 牙E在线
df_yae = deal_with_YAE(df_YE_now,Discount_ye_now)

In [7]:
# 构建查询语句，引用不同的 lichi 变量
query = '''
SELECT i.cinvcode'励齿编码', i.iRefCost'商品档案采购价'
FROM lichi..inventory i
'''
# 执行查询
df_dacgj = pd.read_sql(query, conn)

# 将查询结果与原始DataFrame进行合并
df = pd.merge(df, df_dacgj, left_on='SKU编码', right_on='励齿编码', how='left')

# 选择指定的列
df = df[['SKU编码', '上海名称', '上海类型', '上海到手价格', '商品档案采购价', '上海最后采购价格', '北京最后采购价格', '广州最后采购价格']]

# 重命名列名
new_columns = ['励齿编码', '励齿商品名', '上海励齿是否上架', '上海励齿.折扣价格', '商品档案采购价', '上海最后采购价格', '北京最后采购价格', '广州最后采购价格']
df.columns = new_columns


In [8]:
## 松佰
df_SB = deal_with_SONGBAI(df_SB_now,Discount_SB_now)

## 合并表格

In [9]:
# 定义新的列名
new_columns = ['励齿编码', '梅苗苗编码', '牙E编码', '松佰编码']
df_jp.columns = new_columns

# 将数据框进行合并
df = df.merge(df_jp, on='励齿编码', how='left').merge(df_mmm, on='梅苗苗编码', how='left').merge(df_yae, on='牙E编码', how='left').merge(df_SB, on='松佰编码', how='left')

# 创建副本数据框
dfb = df.copy()

# 删除励齿商品名为空的行
dfb.dropna(subset=['励齿商品名'], inplace=True)

# 计算折扣价格的差异
dfb['励齿/梅苗苗折扣价格比对'] = dfb['上海励齿.折扣价格'] - dfb['梅苗苗.折扣价格']
dfb['励齿/牙E在线折扣价格比对'] = dfb['上海励齿.折扣价格'] - dfb['牙E在线.折扣价格']

# 保留所需的列
dfb = dfb[['励齿编码', '梅苗苗编码', '牙E编码', '松佰编码', '励齿商品名', '梅苗苗品名', '牙E在线品型', '松佰名称', '上海励齿是否上架', '上海励齿.折扣价格', '梅苗苗.折扣价格', '牙E在线.折扣价格', '松佰商城.折扣价格', '励齿/梅苗苗折扣价格比对', '励齿/牙E在线折扣价格比对', '商品档案采购价', '上海最后采购价格', '北京最后采购价格', '广州最后采购价格', '梅苗苗.标签', '松佰商城.标签']]

# 返回结果数据框
dfb


,励齿编码,梅苗苗编码,牙E编码,松佰编码,励齿商品名,梅苗苗品名,牙E在线品型,松佰名称,上海励齿是否上架,上海励齿.折扣价格,...,牙E在线.折扣价格,松佰商城.折扣价格,励齿/梅苗苗折扣价格比对,励齿/牙E在线折扣价格比对,商品档案采购价,上海最后采购价格,北京最后采购价格,广州最后采购价格,梅苗苗.标签,松佰商城.标签
0,310201,1-104785-103789,埃蒙迪 摇椅镍钛方丝弓（弓形4） 10*116*22下,NaN,"埃蒙迪·IMD 摇椅型镍钛超弹方丝 0.016""*0.022""下颌",埃蒙迪 镍钛合金 摇椅型 超弹方丝 10根（2根*5袋）/包0.016*0.022下,埃蒙迪 摇椅镍钛方丝弓（弓形4） 10*116*22下1820002009,NaN,上架,80.00,...,72.20,NaN,0.00,7.80,60.00,60.00,42.86,60.00,NaN,NaN
1,304376,1-112411-119289,NaN,NaN,安萨 牙骨凿 1837-4,安萨/ASA 牙骨凿1837-4,NaN,NaN,上架,204.00,...,NaN,NaN,-1.00,NaN,150.00,150.00,165.00,150.00,NaN,NaN
2,309412,NaN,NaN,NaN,英科 医用乳胶检查手套 无粉标准型 乳白色,NaN,NaN,NaN,下架,73.87,...,NaN,NaN,NaN,NaN,68.00,68.00,22.00,NaN,NaN,NaN
3,303784,1-102771-101972,埃蒙迪 镍钛超弹圆丝（卵圆形/弓形2）10*116下,NaN,"埃蒙迪·IMD 卵圆型镍钛超弹圆丝 0.016""下颌",埃蒙迪 镍钛合金卵圆型超弹圆丝 10根/盒 独立包装0.016下,埃蒙迪 镍钛超弹圆丝（卵圆形/弓形2）10*116下1820018001,NaN,上架,13.00,...,10.64,NaN,0.00,2.36,8.31,10.80,7.71,10.80,NaN,NaN
4,307548,1-109249-112473,NaN,NaN,拓维亚 金属分段式龈上豆瓣成型片补充装 1.0972 0.035mm 不带突,俄罗斯TORVM拓维亚 拓维亚 金属分段式成形片补充装 10片/盒1.0972,NaN,NaN,上架,28.80,...,NaN,NaN,0.00,NaN,24.00,24.00,25.00,24.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32635,LC2486,NaN,NaN,NaN,阿舒尔 牙科橡胶抛光头 BU-W 白色 10/盒+柄*1,NaN,NaN,NaN,下架,33.20,...,NaN,NaN,NaN,NaN,20.00,NaN,NaN,NaN,NaN,NaN
32636,LC2640,NaN,NaN,NaN,(GZ)新亚 标准型直丝弓带环（M系列）UR6 42+# 52c3-20F-142+-10,NaN,NaN,NaN,下架,57.75,...,NaN,NaN,NaN,NaN,57.75,NaN,NaN,53.63,NaN,NaN
32637,300407,1-102718-101841,美丽牙医 37%磷酸酸蚀剂2.5ml2.5ml,NaN,美丽牙医 ETCHANT通用酸蚀剂 37%浓度 2.5ml 磷酸,美丽牙医/MEDENTAL ETCHANT 37%磷酸酸蚀剂（新旧包装混发）2.5ml/支,NaN,NaN,上架,12.01,...,NaN,NaN,1.01,NaN,9.00,9.00,9.00,9.00,NaN,NaN
32638,100091,1-102594-101653,3M 绿巨人树脂水门汀初始装56925透明色56925,NaN,*3M ESPE RelyX Ultimate树脂水门汀（绿巨人） 56925 Clicker,3M RelyX Ultimate 绿巨人树脂水门汀 TR透明色 569254.5g,3M 绿巨人树脂水门汀初始装56925透明色5692530047001,NaN,上架,599.00,...,606.48,NaN,0.00,-7.48,626.29,614.01,626.29,614.01,NaN,NaN


In [10]:
dfb.to_excel(f'C:/Users/15516/Desktop/竞品比价{now}.xlsx',index = False)